# L1C Centering on Latitude

* Option for how many pixels from center it should grab

In [1]:
import pandas as pd
import h5py
# from pyhdf.SD import SD, SDC
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
os.chdir('/project/sdscap-shakeri/nasa/UVA_NASA_2021')
import icare
import pickle
import copy
from netCDF4 import Dataset
import netCDF4
os.getcwd()

'/gpfs/gpfs0/project/sdscap-shakeri/nasa/UVA_NASA_2021'

In [2]:
# os.chdir('C:\\Users\\whetz\\Documents\\UVA MSDS\\NASA\\hdf_files')
# session = icare.ICARESession('/project/sdscap-shakeri/nasa/HDF')

In [3]:
# path='PARASOL/L1_B-HDF.v1.00/2008/2008_06_01'
# file_list = list(session.listdir(path))
# file_path = path + '/' + file_list[0]
# session.get_file(file_path)

In [4]:
os.chdir('/project/sdscap-shakeri/nasa/HDF')

In [5]:
f = h5py.File("PARASOL/L1_B-HDF.v1.00/2008/2008_06_01/POLDER3_L1B-BG1-080146M_2008-06-01T00-08-19_V1-00.h5", "r")

# Precleaning

In [6]:
filt = False
pixels = 10
x1 = 26
x2 = x1+10

In [7]:
i490 = f['Data_Directional_Fields']['I490P']
i490_fill = i490.attrs['_FillValue']
i490_arr = np.array(i490)

print(i490_fill)
i490_arr.shape

32767


(3240, 6480, 16)

In [8]:
3240*6480

20995200

### Cut down extra lat/lons

In [9]:
rows = []
cols = []
for row in range(i490.shape[0]):
    inds = np.where(np.abs(i490_arr[row]) != i490_fill)
    if (len(inds[0])) > 0:
        rows.append(row)
        continue

print(len(rows))

for col in range(i490.shape[1]):
    inds = np.where(np.abs(i490_arr[:,col]) != i490_fill)
    if len(inds[0]) > 0:
        cols.append(col)
        continue
                    
print(len(cols))

2718
5187


In [10]:
new_shape = (i490_arr[rows])[:,cols].shape
shape = i490_arr.shape
print(np.product(shape))
print(np.product(new_shape))
print((np.product(new_shape)-np.product(shape))/np.product(shape))

335923200
225572256
-0.3285005144032922


In [11]:
new_arr = (i490_arr[rows])[:,cols]
np.where(np.abs(new_arr) != i490_fill)

(array([   0,    0,    0, ..., 2716, 2716, 2717]),
 array([2659, 2660, 2661, ..., 1743, 1744, 1743]),
 array([0, 0, 0, ..., 0, 0, 0]))

In [12]:
new_arr[0,2661]

array([  3916, -32767, -32767, -32767, -32767, -32767, -32767, -32767,
       -32767, -32767, -32767, -32767, -32767, -32767, -32767, -32767],
      dtype=int16)

### Lat/Long Exploration

Rows (first dimension) have all the same lats, all different lons

Cols (second dimension) have different lats, all different lons

# Angles, Altitude, & Location

In [13]:
reverse_scale = 1000
new_scale = 1/reverse_scale

In [14]:
final_dict = {}
final_dict['geolocation_data'] = {}

In [15]:
f['Geolocation_Fields'].keys()

<KeysViewHDF5 ['Latitude', 'Longitude', 'column_number', 'land_sea_flag', 'row_number', 'surface_altitude']>

In [16]:
# Get latitude array and corresponding indices
geo_inds = [[],[]]

lats = np.array(f['Geolocation_Fields']['Latitude'])
lons = np.array(f['Geolocation_Fields']['Longitude'])

flag = True
for i in range(lats.shape[0]):
    inds = np.where(np.abs(lats[i]) != 99999)[0]

    vals = lats[i][inds]

    if len(np.unique(vals)) > 1:
        print("Error", i)
        break
    elif len(np.unique(vals)) == 0:
        continue
    else:
        val = np.unique(vals)[0]
    
    center_ind = int(np.sum(inds)/len(inds))

    if center_ind < 1:
        continue
    else:
        temp_lats = lats[i][center_ind-1:center_ind+2] 
        geo_inds[0].extend([i for u in range(3)])
        geo_inds[1].extend([u for u in range(center_ind-1,center_ind+2)])
        
    if flag:
        new_lats = np.array([temp_lats])
        flag = False
    else:
        new_lats = np.append(new_lats, np.array([temp_lats]), axis=0)
        
geo_shape = (new_lats.shape)
print(geo_shape)
len(geo_inds[0])

(2719, 3)


8157

In [17]:
for field in ['Latitude','Longitude','surface_altitude']:
    if field == 'surface_altitude':
        tag = 'altitude'
    else:
        tag = field.lower()
    print(tag)

        
    final_dict['geolocation_data'][tag] = {}

    final_dict['geolocation_data'][tag]['long_name'] = f['Geolocation_Fields'][field].attrs['long_name']
    final_dict['geolocation_data'][tag]['units'] = f['Geolocation_Fields'][field].attrs['units']
    
    if tag in ['latitude','longitude']:
        temp_arr = np.array(f['Geolocation_Fields'][field])
        fill = f['Geolocation_Fields'][field].attrs['_FillValue']
        fill_inds = np.where(temp_arr == fill)
        temp_arr = temp_arr*reverse_scale
        temp_arr[fill_inds] = fill
        fill = int(fill)
        
        final_dict['geolocation_data'][tag]['fill'] = fill
        final_dict['geolocation_data'][tag]['scale'] = new_scale
        final_dict['geolocation_data'][tag]['data'] = np.round(temp_arr[geo_inds].reshape(geo_shape)).astype(int)
    else:
        final_dict['geolocation_data'][tag]['scale'] = f['Geolocation_Fields'][field].attrs['scale_factor']
        final_dict['geolocation_data'][tag]['data'] = (np.array(f['Geolocation_Fields'][field]))[geo_inds].reshape(geo_shape)
        final_dict['geolocation_data'][tag]['fill'] = f['Geolocation_Fields'][field].attrs['_FillValue']


latitude


<ipython-input-17-14aaa9fcb905>:24: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  final_dict['geolocation_data'][tag]['data'] = np.round(temp_arr[geo_inds].reshape(geo_shape)).astype(int)


longitude
altitude


<ipython-input-17-14aaa9fcb905>:27: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  final_dict['geolocation_data'][tag]['data'] = (np.array(f['Geolocation_Fields'][field]))[geo_inds].reshape(geo_shape)


In [18]:
for field in ['thetas','thetav','phi']:
    if field == 'thetas':
        tag = 'solar_zenith'
    elif field == 'thetav':
        tag = 'sensor_zenith'
    elif field == 'phi':
        tag = 'relative_azimuth'
        
    # Define the shape of the data
    field_shape = list(geo_shape)
    field_shape.append(np.array(f['Data_Directional_Fields'][field]).shape[-1])
    field_shape = tuple(field_shape)
        
    final_dict['geolocation_data'][tag] = {}
    final_dict['geolocation_data'][tag]['scale'] = f['Data_Directional_Fields'][field].attrs['scale_factor']
    final_dict['geolocation_data'][tag]['long_name'] = f['Data_Directional_Fields'][field].attrs['long_name']
    final_dict['geolocation_data'][tag]['fill'] = f['Data_Directional_Fields'][field].attrs['_FillValue']
    final_dict['geolocation_data'][tag]['units'] = f['Data_Directional_Fields'][field].attrs['units']
    final_dict['geolocation_data'][tag]['data'] = (np.array(f['Data_Directional_Fields'][field]))[geo_inds].reshape(field_shape)
    print(tag, final_dict['geolocation_data'][tag]['data'].shape)

<ipython-input-18-1d48cb3765a3>:19: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  final_dict['geolocation_data'][tag]['data'] = (np.array(f['Data_Directional_Fields'][field]))[geo_inds].reshape(field_shape)


solar_zenith (2719, 3, 16)
sensor_zenith (2719, 3, 16)
relative_azimuth (2719, 3, 16)


# Generate I, Q, & U fields

In [19]:
measurement_dict = {}
for cat in ['I_np','I_p','Q','U']:
    print(cat)
    
    if cat == 'I_np':
        tag = cat.replace('_np','')
        fields = ([field for field in list(f['Data_Directional_Fields'].keys()) if (tag in field)])
    else:
        tag = cat.replace('_p','')
        fields = ([field for field in list(f['Data_Directional_Fields'].keys()) if (tag in field) and ('NP' not in field)])

    print("tag:", tag)
    fields.sort()
    
    # Define the shape of the data
    field_shape = list(geo_shape)
    field_shape.append(np.array(f['Data_Directional_Fields'][field]).shape[-1])
    field_shape = tuple(field_shape)    
    
    arrays = []
    scales = []
    long_names = []
    fills = []
    units = []
    for field in fields:
            
        print(field, end=", ")
        
        scales.append(f['Data_Directional_Fields'][field].attrs['scale_factor'])
        long_names.append(f['Data_Directional_Fields'][field].attrs['long_name'])
        fills.append(f['Data_Directional_Fields'][field].attrs['_FillValue'])
        units.append(f['Data_Directional_Fields'][field].attrs['units'])
        
        arrays.append(np.array(f['Data_Directional_Fields'][field])[geo_inds].reshape(field_shape))

    if len(np.unique(scales)) == 1:
        scales = scales[0]
    if len(np.unique(fills)) == 1:
        fills = fills[0]
    if len(np.unique(units)) == 1:
        units = units[0]
    
    measurement_dict[cat] = {}
    measurement_dict[cat]['fields'] = fields
    measurement_dict[cat]['scale'] = scales
    measurement_dict[cat]['long_name'] = long_names
    measurement_dict[cat]['fill'] = fills
    measurement_dict[cat]['units'] = units
    measurement_dict[cat]['data'] = np.stack(arrays,axis=3)
    print(measurement_dict[cat]['data'].shape)

I_np
tag: I
I1020NP, 

<ipython-input-19-8d4c14384ad1>:34: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  arrays.append(np.array(f['Data_Directional_Fields'][field])[geo_inds].reshape(field_shape))


I443NP, I490P, I565NP, I670P, I763NP, I765NP, I865P, I910NP, (2719, 3, 16, 9)
I_p
tag: I
I490P, I670P, I865P, (2719, 3, 16, 3)
Q
tag: Q
Q490P, Q670P, Q865P, (2719, 3, 16, 3)
U
tag: U
U490P, U670P, U865P, (2719, 3, 16, 3)


In [20]:
measurement_dict['I_np']['data'].shape

(2719, 3, 16, 9)

# Create Final Dictionary and Populate

In [21]:
final_dict['observation_data'] = {}
final_dict['observation_data']['I_PARASOL'] = copy.deepcopy(measurement_dict['I_np'])

## Wavelengths

In [22]:
final_dict['sensor_views_bands'] = {}

for i in [True, False]:
    if i:
        tag = 'I_p'
        field_name = 'polarization_wavelengths'
        lambdas = [int(field.replace('I','').replace('P','')) for field in measurement_dict[tag]['fields']]

    else:
        tag = 'I_np'
        field_name = 'intensity_wavelengths'
        lambdas = [int(field.replace('I','').replace('NP','').replace('P','')) for field in measurement_dict[tag]['fields']]


    shape = measurement_dict[tag]['data'].shape
    new_shape = []
    lambda_arrs = []
    
    print(field_name)
    
    for lam in lambdas:  
        lambda_arrs.append(np.full(16,lam))

    full_lambdas_arr = np.stack(lambda_arrs,axis=1)
    print(full_lambdas_arr.shape)

    
    final_dict['sensor_views_bands'][field_name] = {}
    final_dict['sensor_views_bands'][field_name]['scale'] = 1
    final_dict['sensor_views_bands'][field_name]['long_name'] = 'field_name'
    final_dict['sensor_views_bands'][field_name]['fill'] = 32767
    final_dict['sensor_views_bands'][field_name]['units'] = 'tbd'
    final_dict['sensor_views_bands'][field_name]['data'] = full_lambdas_arr
    
    print("")

polarization_wavelengths
(16, 3)

intensity_wavelengths
(16, 9)



## DOLP
* Calculate DOLP matrix
    * DOLP = sqrt(Q^2 + U^2)/I

In [23]:
scale = measurement_dict['Q']['scale']
scale
fill = measurement_dict['Q']['fill']
fill

32767

In [24]:
# scale, abs
for key in measurement_dict.keys():
    if key == 'I_p':
        I_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        I_arr[np.abs(I_arr) == fill] = 1
    
    elif key == 'Q':
        Q_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        Q_arr[np.abs(Q_arr) == fill] = 1
        
    elif key == 'U':
        U_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        U_arr[np.abs(U_arr) == fill] = 1
        
    else:
        continue

In [25]:
DOLP_arr_unfltrd = np.divide(np.sqrt(np.add(np.square(Q_arr), np.square(U_arr))), I_arr)*reverse_scale

<ipython-input-25-bca893dafdb9>:1: RuntimeWarning: divide by zero encountered in true_divide
  DOLP_arr_unfltrd = np.divide(np.sqrt(np.add(np.square(Q_arr), np.square(U_arr))), I_arr)*reverse_scale


In [26]:
# rename - DOLP
DOLP_arr_unfltrd[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

In [27]:
indx = np.where((np.abs(I_arr) != (fill * scale)) & (np.abs(Q_arr) != (fill * scale)) & (np.abs(U_arr) != (fill * scale)))

In [28]:
# add to dictionary

final_dict['observation_data']['DOLP_PARASOL'] = {}
# measurement_dict[cat]['fields'] = fields
final_dict['observation_data']['DOLP_PARASOL']['scale'] = new_scale
final_dict['observation_data']['DOLP_PARASOL']['long_name'] = 'Degree of linear polarization'
final_dict['observation_data']['DOLP_PARASOL']['fill'] = fill
final_dict['observation_data']['DOLP_PARASOL']['units'] = 'None'
final_dict['observation_data']['DOLP_PARASOL']['data'] = np.round(DOLP_arr_unfltrd).astype(int)

## Q over I, U over I

In [29]:
Q_over_I = np.divide(Q_arr, I_arr)*reverse_scale
U_over_I = np.divide(U_arr, I_arr)*reverse_scale

Q_over_I[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

U_over_I[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

print("Processing Q over I")
final_dict['observation_data']['Q_over_I_PARASOL'] = {}
final_dict['observation_data']['Q_over_I_PARASOL']['scale'] = new_scale
final_dict['observation_data']['Q_over_I_PARASOL']['long_name'] = 'Q over I'
final_dict['observation_data']['Q_over_I_PARASOL']['fill'] = fill
final_dict['observation_data']['Q_over_I_PARASOL']['units'] = 'None'
final_dict['observation_data']['Q_over_I_PARASOL']['data'] = np.round(Q_over_I).astype(int)

print("Processing U over I")
final_dict['observation_data']['U_over_I_PARASOL'] = {}
final_dict['observation_data']['U_over_I_PARASOL']['scale'] = new_scale
final_dict['observation_data']['U_over_I_PARASOL']['long_name'] = 'U over I'
final_dict['observation_data']['U_over_I_PARASOL']['fill'] = fill
final_dict['observation_data']['U_over_I_PARASOL']['units'] = 'None'
final_dict['observation_data']['U_over_I_PARASOL']['data'] = np.round(U_over_I).astype(int)

Processing Q over I
Processing U over I


<ipython-input-29-63afa7411eca>:1: RuntimeWarning: divide by zero encountered in true_divide
  Q_over_I = np.divide(Q_arr, I_arr)*reverse_scale
<ipython-input-29-63afa7411eca>:2: RuntimeWarning: divide by zero encountered in true_divide
  U_over_I = np.divide(U_arr, I_arr)*reverse_scale


In [30]:
for cat in final_dict.keys():
    for variable in final_dict[cat]:
        print(cat, variable, (final_dict[cat][variable]['data']).dtype)

geolocation_data latitude int64
geolocation_data longitude int64
geolocation_data altitude int16
geolocation_data solar_zenith uint16
geolocation_data sensor_zenith uint16
geolocation_data relative_azimuth uint16
observation_data I_PARASOL int16
observation_data DOLP_PARASOL int64
observation_data Q_over_I_PARASOL int64
observation_data U_over_I_PARASOL int64
sensor_views_bands polarization_wavelengths int64
sensor_views_bands intensity_wavelengths int64


In [31]:
def write_nc(variable_dict, filename):
    
    nc = Dataset(filename, mode='w', format='NETCDF4')

    for cat in variable_dict.keys():
        print("Starting:", cat)
        
        # Create the category group to store the variables
        nc.createGroup(cat)

        for var in variable_dict[cat].keys():
            print(var)

            shape = final_dict[cat][var]['data'].shape
            
            # Fill the dimension with variables
            dimensions = []
            for i in range(len(shape)):  
                dim_name = f'{var}_{i}'
                nc.createDimension(dim_name, size=shape[i])
                dimensions.append(dim_name)

            # Create the variable instance
            print('creating variable')
            nc[cat].createVariable(var, datatype='i8', dimensions=dimensions, fill_value=variable_dict[cat][var]['fill'])

            # Create variable metadata
            print('creating the metadataverse')
            nc[cat][var].long_name = variable_dict[cat][var]['long_name']
            nc[cat][var].units = variable_dict[cat][var]['units']
            nc[cat][var].scale_factor = variable_dict[cat][var]['scale']

            # Create variable array 
            print('creating variable array')
            nc[cat][var][:] = variable_dict[cat][var]['data']
            
            print("")
            
    nc.close()

In [32]:
os.chdir('/gpfs/gpfs0/project/sdscap-shakeri/nasa/UVA_NASA_2021')

In [33]:
write_nc(final_dict, 'example_centered.nc')

Starting: geolocation_data
latitude
creating variable
creating the metadataverse
creating variable array

longitude
creating variable
creating the metadataverse
creating variable array

altitude
creating variable
creating the metadataverse
creating variable array

solar_zenith
creating variable
creating the metadataverse
creating variable array

sensor_zenith
creating variable
creating the metadataverse
creating variable array

relative_azimuth
creating variable
creating the metadataverse
creating variable array

Starting: observation_data
I_PARASOL
creating variable
creating the metadataverse
creating variable array

DOLP_PARASOL
creating variable
creating the metadataverse
creating variable array

Q_over_I_PARASOL
creating variable
creating the metadataverse
creating variable array

U_over_I_PARASOL
creating variable
creating the metadataverse
creating variable array

Starting: sensor_views_bands
polarization_wavelengths
creating variable
creating the metadataverse
creating variable 

In [34]:
nc = Dataset('example_centered.nc', mode='r', format='NETCDF4')

In [35]:
i = np.array(nc.groups['observation_data']['I_PARASOL'])
np.where(i != 32767)

(array([   0,    0,    0, ..., 2718, 2718, 2718]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([ 0,  0,  0, ..., 15, 15, 15]),
 array([0, 1, 2, ..., 6, 7, 8]))

In [36]:
np.array(nc.groups['geolocation_data']['longitude'])

array([[ -99999.,  109125.,  111375.],
       [ 100843.,  103012.,  105181.],
       [  97326.,   99419.,  101512.],
       ...,
       [-156992., -156873., -156753.],
       [-157066., -156946., -156826.],
       [ -99999., -157140.,  -99999.]])

In [61]:
I_PARASOL = final_dict['observation_data']['I_PARASOL']['data']
np.where(np.abs(I_PARASOL) != 32767)

(array([   0,    0,    0, ..., 2717, 2717, 2718]),
 array([1, 1, 2, ..., 1, 2, 1]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([3, 4, 3, ..., 2, 2, 2]))